In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import os
import csv
import sklearn
import matplotlib.image as mpimg
import keras 

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
!unzip ./drive/My\ Drive/data-transfer.zip -d .

In [54]:
model = keras.models.load_model('./drive/My Drive/model.h5', custom_objects={'tf': tf, 'resize_shape': 128})

In [55]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_3 (Cropping2D)    (None, 85, 320, 3)        0         
_________________________________________________________________
lambda_5 (Lambda)            (None, 85, 320, 3)        0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 124, 124, 16)      1216      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 58, 58, 32)        1283

In [56]:
for i in range(6):
  model.layers[i].trainable = False

In [57]:
# original shape of the image
input_shape = (160,320,3)

# first input layer of the model
car_input = keras.layers.Input(shape=input_shape)

mod = model(car_input)

fc1 = keras.layers.Dense(2048, activation=tf.nn.relu)(model.layers[-5].output)

dropout2 = keras.layers.Dropout(.5)(fc1)

fc2 = keras.layers.Dense(1024, activation=tf.nn.relu)(dropout2)

dropout3 = keras.layers.Dropout(.5)(fc2)

fc3 = keras.layers.Dense(512, activation=tf.nn.relu)(dropout3)

dropout4 = keras.layers.Dropout(.5)(fc3)

fc4 = keras.layers.Dense(256, activation=tf.nn.relu)(dropout4)

prediction = keras.layers.Dense(1)(fc4)


In [63]:
new_model = keras.models.Model(inputs=model.get_input_at(0), output=prediction)

new_model.summary()
new_model.compile(optimizer='Adam', loss='mse')


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_3 (Cropping2D)    (None, 85, 320, 3)        0         
_________________________________________________________________
lambda_5 (Lambda)            (None, 85, 320, 3)        0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 124, 124, 16)      1216      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 58, 58, 32)        128

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [69]:
# Create generator for dataset with images and steering measurements
from sklearn.model_selection import train_test_split

# path where the data is stored 
path = './data-transfer'

def get_samples(path):
  # read in the csv lines 
  samples = []
  with open(path+'/driving_log.csv') as file:
      reader = csv.reader(file)
      for line in reader: 
          samples.append(line)

  # split csv lines into test, validation, and test
  train_samples, valid_samples = train_test_split(samples, test_size=.2)
  return train_samples, valid_samples


In [80]:
def generator(samples, path, batch_size=32):
    while True: 
        sklearn.utils.shuffle(samples)
        # for each batch
        for start in range(0, len(samples), batch_size//2): # batch size is divided by 2 because we add the flipped images as well
            batch_samples = samples[start:start+batch_size]

            images = []
            angles = []

            # for each csv line
            for batch_sample in batch_samples:
                
                # get image path
                name = path+'/IMG/'+batch_sample[0].split('\\')[-1]
                
                # add image and angle
                center_image = mpimg.imread(name)
                center_angle=float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                
                # flip image 
                center_image_flip = np.fliplr(center_image)
                center_angle_flip = -center_angle
                images.append(center_image_flip)
                angles.append(center_angle_flip)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [81]:
batch_size = 32
from math import ceil

train_samples, valid_samples = get_samples(path)

train_generator = generator(train_samples, path, batch_size=batch_size)
valid_generator = generator(valid_samples, path, batch_size=batch_size)

In [72]:
history = new_model.fit_generator(train_generator, 
            steps_per_epoch=ceil(len(train_samples)/batch_size), 
            validation_data=valid_generator, 
            validation_steps=ceil(len(valid_samples)/batch_size), 
            epochs=10, verbose=1)

#new_model.save('model-transfer2.h5')

Epoch 1/10
301/301 [==============================] - 23s 77ms/step - loss: 0.0020 - val_loss: 2.8376e-04
Epoch 2/10
301/301 [==============================] - 22s 75ms/step - loss: 0.0020 - val_loss: 0.0017
Epoch 3/10
301/301 [==============================] - 22s 75ms/step - loss: 0.0016 - val_loss: 0.0023
Epoch 4/10
301/301 [==============================] - 22s 74ms/step - loss: 0.0017 - val_loss: 0.0073
Epoch 5/10
301/301 [==============================] - 23s 75ms/step - loss: 0.0016 - val_loss: 0.0033
Epoch 6/10
301/301 [==============================] - 23s 75ms/step - loss: 0.0018 - val_loss: 0.0027
Epoch 7/10
301/301 [==============================] - 23s 75ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 8/10
301/301 [==============================] - 22s 74ms/step - loss: 0.0017 - val_loss: 3.9944e-04
Epoch 9/10
301/301 [==============================] - 23s 76ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 10/10
301/301 [==============================] - 22s 74ms/step - los

In [82]:
# run transfer-learning on another set of data 
new_path = './data4'

In [83]:
train_samples, valid_samples = get_samples(new_path)

train_generator = generator(train_samples, new_path, batch_size=batch_size)
valid_generator = generator(valid_samples, new_path, batch_size=batch_size)

history = new_model.fit_generator(train_generator, 
            steps_per_epoch=ceil(len(train_samples)/batch_size), 
            validation_data=valid_generator, 
            validation_steps=ceil(len(valid_samples)/batch_size), 
            epochs=30, verbose=1)

new_model.save('model-transfer3.h5')

Epoch 1/30
123/123 [==============================] - 10s 78ms/step - loss: 0.0055 - val_loss: 0.0047
Epoch 2/30
123/123 [==============================] - 9s 72ms/step - loss: 0.0052 - val_loss: 7.1433e-04
Epoch 3/30
123/123 [==============================] - 9s 74ms/step - loss: 0.0045 - val_loss: 0.0039
Epoch 4/30
123/123 [==============================] - 9s 73ms/step - loss: 0.0044 - val_loss: 0.0012
Epoch 5/30
123/123 [==============================] - 9s 74ms/step - loss: 0.0040 - val_loss: 0.0055
Epoch 6/30
123/123 [==============================] - 9s 74ms/step - loss: 0.0041 - val_loss: 0.0012
Epoch 7/30
123/123 [==============================] - 9s 74ms/step - loss: 0.0035 - val_loss: 0.0059
Epoch 8/30
123/123 [==============================] - 9s 73ms/step - loss: 0.0035 - val_loss: 5.5636e-04
Epoch 9/30
123/123 [==============================] - 9s 74ms/step - loss: 0.0031 - val_loss: 0.0064
Epoch 10/30
123/123 [==============================] - 9s 73ms/step - loss: 0.0032